In [42]:
import numpy as np
import pandas as pd

all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/1887271186.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [44]:
all_data.columns


Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [45]:
# 注意一下，这里的列名和huili最终的sample中的列名有一点大小写区别
cv_columns = ['cv_id', 'currentPosition', 'desiredPosition', 'industry', 'desiredIndustry', 'majorName', 'skills', 'eduTracks', 'jobTracks', 'projectTracks']
jd_columns = ['jd_id', 'title', 'category_name', 'tags', 'description', 'requirement']

In [46]:
jd_data = all_data[jd_columns]
jd_data

# 可以做3个向量
# 1.title + category_name + tags
# 2.description
# 3.requirement
# 每个向量的维度在20维左右



,jd_id,title,category_name,tags,description,requirement
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...
...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...


In [47]:
# 将title + category_name + tags合并成一个字段

# step1: 缺失值填充
jd_data['title'].fillna('', inplace=True)
jd_data['category_name'].fillna('', inplace=True)
jd_data['tags'].fillna('[]', inplace=True)


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/207131757.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/207131757.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['category_name'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/207131757.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [48]:
# step2: 定义新列，将title + category_name合并
def merge_title_category_name_fun(series):
    title = series['title']
    category_name = series['category_name']
    return title + ' ' + category_name

jd_data['title_category'] = jd_data.apply(merge_title_category_name_fun, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/1919854961.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category'] = jd_data.apply(merge_title_category_name_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理
...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java


In [49]:
# step3 将新列title_category通过jieba分词处理成列表
import re
import json
import jieba

def col_jieba_fun(series):
    col = series['title_category']

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/2706112485.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]"
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]"
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]"
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]"
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]"
...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]"
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]"
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]"
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]"


In [50]:
# step4 将新列tags通过jieba分词处理成列表
import re
import json
import jieba

def col_jieba_fun(series):
    col = series['tags']

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/2841311902.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]"
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]"
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]"
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]"
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]"
...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]"
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]"
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]"
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ..."


In [ ]:
# step5 将title_category_jieba和tags_jieba两个列表合并
def merge_title_category_tags_jieba(series):
    title_category_jieba = series['title_category_jieba']
    tags_jieba = series['tags_jieba']
    return title_category_jieba + tags_jieba

jd_data['title_category_tags_jieba'] = jd_data.apply(merge_title_category_tags_jieba, axis=1)
jd_data

In [ ]:
#step6 过滤title_category_tags_jieba, 得到以空格分割的字符串
from zhconv import convert

def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series['title_category_tags_jieba']
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

In [ ]:
jd_data[['title_category_tags_jieba', 'title_category_tags_jieba_filter']]

In [ ]:
#step7 将title_category_tags_jieba_filter转成tfidf向量
# 转成tfidf向量
def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'title_category_tags_jieba_filter')
tfidf


In [ ]:
vectorizer.vocabulary_

In [ ]:
# 小插曲 
# 'blockchainindustryresearcher': 103,
for index, value in enumerate(tfidf[103]):
    if value:
        print("{}: {}".format(index, value))

# 显示的这几条数据都是来源于同一个jd
# 但为何有不同的jd_id呢？

In [ ]:
jd_data.iloc[47272]

In [ ]:
#step8 对tfidf降维到n维
def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

In [ ]:
jd_data.info()

In [ ]:
jd_data

In [ ]:
cv_data = all_data[cv_columns].drop(['industry', 'desiredIndustry'], axis=1)
cv_data
'''
majorName字段是不是不太可信
eduTracks放弃

可以做如下4个向量：
1.currentPosition + desiredPosition
2.skills
3.jobTracks
4.projectTracks
'''

In [ ]:
cv_data.info()

In [ ]:
cv_data['eduTracks'][0]

In [ ]:
cv_data['jobTracks'][1]

In [ ]:
cv_data['projectTracks'][0]

In [ ]:
df=pd.DataFrame({
    'a':pd.Series([np.NaN, np.nan, pd.NaT, '', None, 'I stay']),'b':pd.Series([np.NaN, 2, 3, 4, None, 6])})
df

In [ ]:
# 原来np.NaN, pd.NaT和None都会被判为空值。但是''不会被判为空值。
df.isnull()

In [ ]:
# 深刻理解axis
df.isnull().sum(axis=0)